<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Neural_Architecture_Search_(NAS).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import torch
import torch.nn as nn
import torch.optim as optim

class RandomNet(nn.Module):
    def __init__(self, layer_sizes):
        super(RandomNet, self).__init__()
        layers = []
        for i in range(len(layer_sizes) - 1):
            layers.append(nn.Linear(layer_sizes[i], layer_sizes[i+1]))
            if i < len(layer_sizes) - 2:
                layers.append(nn.ReLU())
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

def random_search(input_dim, output_dim, search_space, train_data, train_labels, epochs=5):
    best_model = None
    best_loss = float('inf')

    for i in range(search_space['num_models']):
        layer_sizes = [input_dim] + random.sample(search_space['hidden_sizes'], k=random.randint(1, search_space['max_layers'])) + [output_dim]
        model = RandomNet(layer_sizes)
        optimizer = optim.Adam(model.parameters(), lr=0.001)

        for epoch in range(epochs):
            optimizer.zero_grad()
            output = model(train_data)
            loss = nn.MSELoss()(output, train_labels)
            loss.backward()
            optimizer.step()

        if loss.item() < best_loss:
            best_loss = loss.item()
            best_model = model

    return best_model, best_loss

# Example usage
search_space = {
    'num_models': 10,
    'hidden_sizes': [16, 32, 64, 128],
    'max_layers': 3
}

train_data = torch.randn(100, 10)
train_labels = torch.randn(100, 1)

best_model, best_loss = random_search(10, 1, search_space, train_data, train_labels)
print(f'Best Loss: {best_loss:.4f}')